In [ ]:
pip install openpyxl

In [1]:
import database, functions

In [2]:
energy_engine =database.connect_to_db('energy')

In [3]:
energy_df = functions.read_raw_energy_data()
database.df_to_mysql_table(energy_df, 'energy_data_raw', energy_engine)
codebook_df = functions.read_raw_energy_codebook()
database.df_to_mysql_table(codebook_df, 'codebook_raw', energy_engine)
country_info_df = functions.read_raw_country_info()
database.df_to_mysql_table(country_info_df, 'country_info_raw', energy_engine)

La tabla 'energy_data_raw' se ha insertado exitosamente en la base de datos.
La tabla 'codebook_raw' se ha insertado exitosamente en la base de datos.
La tabla 'country_info_raw' se ha insertado exitosamente en la base de datos.


In [4]:
energy_df_cleaned = functions.drop_aggregated_data(energy_df, ['iso_code', 'country', 'year'])
config_df = functions.read_config()

energy_df_cleaned = functions.keep_and_rename_columns(energy_df_cleaned, config_df)

In [5]:
energy_df_cleaned.sort_values(by=['País', 'Año'], inplace=True)
energy_df_cleaned = functions.interpolate_missing_data(energy_df_cleaned, 'País', 'PIB (Producto Interno Bruto)')
energy_df_cleaned = functions.datatype_conversion(energy_df_cleaned)
database.df_to_mysql_table(energy_df_cleaned, 'energy_data', energy_engine)

La tabla 'energy_data' se ha insertado exitosamente en la base de datos.


In [ ]:
functions.dataset_info(energy_df_cleaned)

In [ ]:
functions.plot_data(energy_df_cleaned)

In [7]:
# SQL query
query = """
SELECT *
FROM `energy`.`energy_data` AS e
JOIN (
    SELECT Region AS `Región`, LifeExpectancy AS `Expectativa de vida`, Code
    FROM world.country
) AS w ON e.`Código ISO` = w.`Code`
JOIN (
    SELECT name_es AS `País ES`, continent_es AS Continente, km2 AS `Superficie, km2`, code_3 
    FROM energy.country_info_raw
) AS c ON e.`Código ISO` = c.code_3;
"""

df = database.select_data_for_powerbi(energy_engine, query)

if df.empty:
    print("La consulta no resultó exitosa")

excel_filename = './cleaned/energy_data.xlsx'
df.to_excel(excel_filename, index=False)
print(f"Data saved to {excel_filename}")


Data saved to ./cleaned/energy_data.xlsx
